In [1]:
import zipfile
import pandas as pd
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from numpy import average as avg
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

# nltk.download('punkt')
# nltk.download('omw-1.4')
# nltk.download('stopwords')

# Read Training Data
Data source: https://www.kaggle.com/datasets/kazanova/sentiment140  

Columns:
1. label:   The polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
2. id:      The id of the tweet
3. date:     The date of the tweet (Sat May 16 23:58:44 UTC 2009)
4. flag:     The query (lyx). If there is no query, then this value is NO_QUERY.
5. user:     The user that tweeted
6. raw_text: The text of the tweet

In [2]:
archive = zipfile.ZipFile("data.zip", "r")
data = pd.read_csv(archive.open("train_binary.csv"), header=None, encoding_errors="replace")
data.columns = ["label", "id", "date", "flag", "user", "raw_text"]
data

,label,id,date,flag,user,raw_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


Number of tweets per label:

In [3]:
data["label"].value_counts()

0    800000
4    800000
Name: label, dtype: int64

There are no neutral labelled tweets.
So we change the labels to binary form.
(0 = negative
1 = positive)

In [4]:
data.loc[data["label"]==4, "label"] = 1

# Data Preprocessing

Turning everything into lowercase characters

In [5]:
data["text"] = [entry.lower() for entry in data["raw_text"]]
data.head()

,label,id,date,flag,user,raw_text,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,is upset that he can't update his facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,@kenichan i dived many times for the ball. man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","@nationwideclass no, it's not behaving at all...."


Removing Stopwords (Common words like "my", "he", "is", ...)

In [6]:
stop = stopwords.words('english')
# we think 'no' and 'not' might be important words for the sentiment and don't want them to be removed,
# so we remove them from the list of stopwords
stop.remove("no")
stop.remove("not")

Removing links, tags and several punctuations from tweets using regular expressions

In [7]:
data["text"] = data["text"].apply(lambda x: re.sub("http[s]?://\S+", "", x))
data["text"] = data["text"].apply(lambda x: re.sub("@\S+", "", x))
data["text"] = data["text"].apply(lambda x: re.sub("[^a-z\s]", "", x))

data.head()

,label,id,date,flag,user,raw_text,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww thats a bummer you shoulda got david ...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,is upset that he cant update his facebook by t...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball managed to sa...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i he...


Word-Stemming  
This did not have the desired effect, but made the model worse

In [8]:
sstemmer = SnowballStemmer("english")
# pstemmer = PorterStemmer()

data['text'] = data['text'].apply(lambda x: ' '.join([sstemmer.stem(word) for word in x.split()]))

Tokenize Words (transform sentence into list of words)

In [9]:
# This process takes a few minutes. So you only need to do it once, save it to a csv-file and read that file later
### comment these lines after running once:
data["text"] = [str(word_tokenize(entry)) for entry in data["text"]]
data.to_csv('tokenized_data.csv', index=False)
###

data = pd.read_csv('tokenized_data.csv')

Our finished data:

In [10]:
data.head()

,label,id,date,flag,user,raw_text,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","['awww', 'that', 'a', 'bummer', 'you', 'should..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,"['is', 'upset', 'that', 'he', 'cant', 'updat',..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,"['i', 'dive', 'mani', 'time', 'for', 'the', 'b..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,"['my', 'whole', 'bodi', 'feel', 'itchi', 'and'..."
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","['no', 'it', 'not', 'behav', 'at', 'all', 'im'..."


# Model Training

Split Training/Test-Data and transform into a vectorized form

In [14]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data["text"], data["label"], test_size=0.2, random_state=10)

In [15]:
Tfidf_vect = TfidfVectorizer(analyzer="word", strip_accents="unicode", stop_words=stop, min_df=10)
# Data_Tfidf = Tfidf_vect.fit_transform(data["text"])
X_train_Tfidf = Tfidf_vect.fit_transform(X_train)
X_test_Tfidf = Tfidf_vect.transform(X_test)

In [16]:
print("# of Features (Words):", len(Tfidf_vect.get_feature_names_out()))

# of Features (Words): 23527


Naive Bayes Classifier with cross-validation

In [14]:
NB = naive_bayes.MultinomialNB()
xval = model_selection.cross_validate(NB, Data_Tfidf, data["label"], cv=10)

print(f"Average Accuracy of X-Val: {round(avg(xval['test_score'])*100, 2)}%")

# fit model on all data
NB.fit(Data_Tfidf, data["label"])

Average Accuracy of X-Val: 76.3%


MultinomialNB()

Support Vector Machine

We did not use GridSearch here, because one iteration of training takes about 20 minutes on our local computers.  
Some manual hyperparameter-tuning was done and the parameters below give the best result we could find.
  
We also need to set the max_iter parameter for the training to not run endlessly. You can adjust the cache_size (used RAM in MB) and max_iter according to your hardware.

In [18]:
# Beacuse of the large dataset this can take a few minutes
SVM = svm.SVC(C=1, kernel='rbf', degree=3, gamma='auto', cache_size=2000, max_iter=2500, decision_function_shape="ovo")
SVM.fit(X_train_Tfidf, y_train)

predictions_SVM = SVM.predict(X_test_Tfidf)
print(f"SVM Accuracy Score: {round(accuracy_score(predictions_SVM, y_test)*100, 2)}%")

c:\Users\flori\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=2500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM Accuracy Score: 56.34%


# Test on self-labelled game related tweets

In [ ]:
testdata = pd.read_excel("testdata_games.xlsx")
testdata = testdata[["text", "multiclass_label"]]
testdata = testdata.loc[testdata["multiclass_label"] != 1]
testdata = testdata.dropna()
testdata["multiclass_label"] = testdata["multiclass_label"].astype(int)
testdata.loc[testdata["multiclass_label"]==2, "multiclass_label"] = 1

testdata

In [ ]:
testdata["multiclass_label"].value_counts()

In [ ]:
testdata_tfidf = Tfidf_vect.transform(testdata["text"])

In [ ]:
predictions_NB = NB.predict(testdata_tfidf)
# predictions_NB = pd.Series(predictions_NB)

In [ ]:
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, testdata["multiclass_label"])*100)